# Step 1: Import helpful libraries

In [3]:
#Load the librarys
import pandas as pd #To work with dataset
import numpy as np #Math library
import seaborn as sns #Graph library that use matplot in background
import matplotlib.pyplot as plt #to plot some parameters in seaborn
from scipy import sparse
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.neighbors import KNeighborsClassifier
# Import StandardScaler from scikit-learn
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
from sklearn.impute import SimpleImputer
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import make_column_transformer,ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.manifold import TSNE
from sklearn.model_selection import GridSearchCV
# Import train_test_split()
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from datetime import datetime, date
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.model_selection import cross_val_score
import lightgbm as lgbm
from catboost import CatBoostRegressor
import  tensorflow as tf 
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
#import smogn
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
# For training random forest model
import lightgbm as lgb
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
#import smong 
from sklearn.linear_model import LinearRegression, RidgeCV
import category_encoders as ce
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings('ignore')

KeyboardInterrupt: 


# Step 2: Load the data

Next, we'll load the training and test data.

We set index_col=0 in the code cell below to use the id column to index the DataFrame. (If you're not sure how this works, try temporarily removing index_col=0 and see how it changes the result.)


In [ ]:
# import lux
# Load the training data
train = pd.read_csv("../input/30-days-of-ml/train.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")
# Preview the data
train.head()

In [ ]:
train.describe()

In [ ]:
# Check the structure of the data
print(train.info())

# Convert Dtypes : 

In [ ]:
train[train.select_dtypes(['float64']).columns] = train[train.select_dtypes(['float64']).columns].apply(pd.to_numeric)
train[train.select_dtypes(['object']).columns] = train.select_dtypes(['object']).apply(lambda x: x.astype('category'))

In [ ]:
train.info()

# Define the model features and target
## Extract X and y 

In [2]:
# Create arrays for the features and the response variable
y = train['target'].to_numpy()
X = train.drop(['id','target'], axis=1)

NameError: name 'train' is not defined

# Create test and train groups

Now we’ve got our dataframe ready we can split it up into the train and test datasets for our model to use. We’ll use the Scikit-Learn train_test_split() function for this. By passing in the X dataframe of raw features, the y series containing the target, and the size of the test group (i.e. 0.1 for 10%), we get back the X_train, X_test, y_train and y_test data to use in the model.

In [ ]:
# Split the dataset and labels into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=0)
print("{} rows in test set vs. {} in training set. {} Features.".format(X_test.shape[0], X_train.shape[0], X_test.shape[1]))

# What should we do for each colmun
## Separate features by dtype

Next we’ll separate the features in the dataframe by their datatype. There are a few different ways to achieve this. I’ve used the select_dtypes() function to obtain specific data types by passing in np.number to obtain the numeric data and exclude=['np.number'] to return the categorical data. Appending .columns to the end returns an Index list containing the column names. For the categorical features, we don’t want to include the target income column, so I’ve dropped that.
## Cat Features 

In [ ]:
# select non-numeric columns
cat_columns = train.drop(['id','target'], axis=1).select_dtypes(exclude=['int64','float64']).columns

## Num Features 

In [ ]:
# select the float columns
num_columns = train.drop(['id','target'], axis=1).select_dtypes(include=['int64','float64']).columns

In [ ]:
print(cat_columns)
print(num_columns)

# check that we have all column

In [ ]:
num_columns=['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7',
       'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13']
cat_columns=['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8',
       'cat9']
all_columns = (num_columns+cat_columns)
print(all_columns)

In [ ]:
if set(all_columns) == set(train.drop(['id','target'], axis=1).columns):
    print('Ok')
else:
    # Let's see the difference 
    print('dans all_columns mais pas dans train  :', set(all_columns) - set(train.drop(['id','target'], axis=1).columns))
    print('dans X.columns   mais pas dans all_columns :', set(train.drop(['id','target'], axis=1).columns) - set(all_columns))

# Preprocess Cat that occur rarely 
## What is high cardinality?

Almost all datasets now have categorical variables. Each categorical variable consists of unique values. A categorical feature is said to possess high cardinality when there are too many of these unique values. One-Hot Encoding becomes a big problem in such a case since we have a separate column for each unique value (indicating its presence or absence) in the categorical variable. This leads to two problems, one is obviously space consumption, but this is not as big a problem as the second problem, the curse of dimensionality.
## The Curse of Dimensionality

Here is a simple summarization:

    As the number of features grows, the amount of data we need to accurately be able to distinguish between these features (in order to give us a prediction) and generalize our model (learned function) grows EXPONENTIALLY.   
would like to use Yoshua Bengio’s (Yes the legendary Yoshua Bengio !) quora answer to explain this in more detail. I strongly advise reading the whole answer here. According to the answer, increasing the number of different values in a feature simply increases the total number of possible combinations that can be made using the input row (containing n such features). Say we have two features with two distinct values each, this gives us a total of 4 possible ways to combine the two features. Now if one of these had three distinct values we would have 3X2 =6 possible ways to combine them.

In classical non-parametric learning algorithms (e.g. nearest-neighbor, Gaussian kernel SVM, Gaussian kernel Gaussian Process, etc.) the model needs to see at least one example for each of these combinations (or at least as many as necessary to cover all the variations of configurations of interest), in order to produce a correct answer, one that is different from the target value required for other nearby configurations.

There is a workaround to this, that is the model even in the absence of a lot of training data can discern between configurations (not in the training set) for future predictions provided there is some sort of structure (pattern) in these combinations. In most cases, high cardinality makes it difficult for the model to identify such patterns and hence the model doesn’t generalise well to examples outside the training set.    
## Reducing Cardinality by using a simple Aggregating function

Below is a simple function I use to reduce the cardinality of a feature. The idea is very simple. Leave instances belonging to a value with high frequency as they are and replace the other instances with a new category which we will call other.

    Choose a threshold
    Sort unique values in the column by their frequency in descending order
    Keep adding the frequency of these sorted (descending) unique values until a threshold is reached.
    These are the unique categories we will keep and instances of all other categories shall be replaced by “other”.
Let’s run through a quick example before going through the code. Say our column colour has 100 values and our threshold is 90% (that is 90). We have 5 different categories of colours: Red (50), Blue(40), Yellow (5), Green (3) and Orange (2). The numbers within the bracket indicate how many instances of that category are present in the column.

We see that Red (50)+Blue (40) reaches our threshold of 90. In that case, we retain only 2 categories (Red, Blue) and mark all other instances of other colours as “Other”
Thus we have reduced cardinality from 5 to 3 (Red, Blue, Other)

Here is the utility function I wrote to facilitate this. It’s well commented and follows exactly what I described above so you won’t have a problem following along. We can set a custom threshold and the return_categories option optionally lets us see the list of all unique values after reducing cardinality.
    

In [ ]:
# Create a series out of the Country column
cat6 = train.cat6

# Get the counts of each category
cat6_counts = cat6.value_counts()

# Print the count values for each category
print(cat6_counts)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="darkgrid")
sns.barplot(cat6_counts.index, cat6_counts.values, alpha=0.9)
plt.title('Frequency Distribution of countries')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Country', fontsize=12)
plt.show()

In [ ]:
labels = train['cat7'].astype('category').cat.categories.tolist()
counts = train['cat7'].value_counts()
sizes = [counts[var_cat] for var_cat in labels]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=True) #autopct is show the % on plot
ax1.axis('equal')
plt.show()

In [ ]:
for col in cat_columns :
    cat_frenquency = (train[col].value_counts())/train.shape[0]
    botton_decile = cat_frenquency.quantile(q=0.05)
    print(col , cat_frenquency,botton_decile )   

# Condense Data with Threshold of 0.05

In [ ]:
def condense_category(col, min_freq=0.05, new_name='other'):
    series = pd.value_counts(col)
    mask = (series/series.sum()).lt(min_freq)
    return pd.Series(np.where(col.isin(series[mask].index), new_name, col))
train_condense=train.copy()
train_condense[cat_columns]=train_condense[cat_columns].apply(condense_category, axis=0)

In [ ]:
train_condense.info()

##  Convert Ddtype

In [ ]:
train_condense[train_condense.select_dtypes(['float64']).columns] = train_condense[train_condense.select_dtypes(['float64']).columns].apply(pd.to_numeric)
train_condense[train_condense.select_dtypes(['object']).columns] = train_condense.select_dtypes(['object']).apply(lambda x: x.astype('category'))

In [ ]:
for col in cat_columns :
    cat_frenquency = (train_condense[col].value_counts())
    botton_decile = cat_frenquency.quantile(q=0.05)
    print(col , cat_frenquency,botton_decile )   

In [ ]:
# Create a series out of the Country column
cat6 = train_condense.cat6

# Get the counts of each category
cat6_counts = cat6.value_counts()

# Print the count values for each category
print(cat6_counts)

In [ ]:
labels = train_condense['cat7'].astype('category').cat.categories.tolist()
counts = train_condense['cat7'].value_counts()
sizes = [counts[var_cat] for var_cat in labels]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=True) #autopct is show the % on plot
ax1.axis('equal')
plt.show()


##  X and Y for data Condense 

In [ ]:
# Create arrays for the features and the response variable
y_condense = train_condense['target'].to_numpy()
X_condense = train_condense.drop(['id','target'], axis=1)
# Split the dataset and labels into training and test sets
X_train_condense , X_test_condense , y_train_condense , y_test_condense  = train_test_split(X_condense , y_condense , test_size=0.1,random_state=0)
print("{} rows in test set vs. {} in training set. {} Features.".format(X_test_condense.shape[0], X_train_condense.shape[0], X_test_condense.shape[1]))

# Condense Data with Threshold of 0.1

In [ ]:
def condense_category(col, min_freq=0.1, new_name='other'):
    series = pd.value_counts(col)
    mask = (series/series.sum()).lt(min_freq)
    return pd.Series(np.where(col.isin(series[mask].index), new_name, col))
train_condense2=train.copy()
train_condense2[cat_columns]=train_condense2[cat_columns].apply(condense_category, axis=0)
train_condense2[train_condense2.select_dtypes(['float64']).columns] = train_condense2[train_condense2.select_dtypes(['float64']).columns].apply(pd.to_numeric)
train_condense[train_condense2.select_dtypes(['object']).columns] = train_condense2.select_dtypes(['object']).apply(lambda x: x.astype('category'))
# Create arrays for the features and the response variable
y_condense2 = train_condense2['target'].to_numpy()
X_condense2 = train_condense2.drop(['id','target'], axis=1)
# Split the dataset and labels into training and test sets
X_train_condense2 , X_test_condense2 , y_train_condense2 , y_test_condense2  = train_test_split(X_condense2 , y_condense2 , test_size=0.1,random_state=0)
print("{} rows in test set vs. {} in training set. {} Features.".format(X_test_condense2.shape[0], X_train_condense2.shape[0], X_test_condense2.shape[1]))

# Create complexe transformer  in order to  put all transformations in the same pipe 
    'num_columns' :Cleaning->Valeur Manquante -> Standar_Scaler
    'cat_columns' : Cleaning -> Valeur Manquante -> Categorique [One Hot]


    fill_missing_then_Standar_scaler = make_pipeline( SimpleImputer(strategy='median',add_indicator=True),
        StandardScaler()
    )

    "Write a pattern to extract numbers and decimals"
    def return_number(string):
        pattern = re.compile(r"\d+\.\d+")
        # Search the text for matches
        number = re.match(pattern, string)
        # If a value is returned, use group(0) to return the found value
        if number is not None:
            return float(number.group(0))

    extraire_number_then_imput_then_scale = make_pipeline(
        FunctionTransformer(extract_number),
        fill_missing_then_Standar_scaler,
    )    


# Pipe Cat 

In [ ]:
fill_missing_then_one_hot_encoder = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='manquante',add_indicator=True),
    OneHotEncoder(handle_unknown='ignore')
)
fill_missing_then_one_hot_encoder1 = make_pipeline(
    SimpleImputer(strategy='most_frequent', add_indicator=True),
    OneHotEncoder(handle_unknown='ignore')
)
fill_missing_then_one_hot_LabelEncoder = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='manquante',add_indicator=True),
    LabelEncoder()
)
fill_missing_then_OrdinalEncoder = make_pipeline(
    SimpleImputer(strategy='most_frequent', fill_value='manquante',add_indicator=True),
    OrdinalEncoder()
)

## Pipe Num  :

In [ ]:
fill_missing_then_StandardScaler = make_pipeline( SimpleImputer(strategy='median',add_indicator=True),
    StandardScaler()
)
fill_missing_then_StandardScalerPolynomialFeatures = make_pipeline( SimpleImputer(strategy='median',add_indicator=True),
     PolynomialFeatures(degree=2),StandardScaler()
)
fill_missing_then_RobustScaler = make_pipeline( SimpleImputer(strategy='median',add_indicator=True),
    RobustScaler()
)
fill_missing_then_MinMaxScaler = make_pipeline( SimpleImputer(strategy='median',add_indicator=True),
    MinMaxScaler()
)
fill_missing_then_Outlier_MinMax = make_pipeline( SimpleImputer(strategy='median',add_indicator=True),
    MinMaxScaler()
) 

# Compose num+cat : ColumnTransformer
## First Standar preprocess

In [ ]:
data_preprocessOrdinalEncoder = make_column_transformer(
    ( fill_missing_then_OrdinalEncoder , cat_columns),
    ( fill_missing_then_StandardScaler, num_columns)
)
data_preprocessone_hot_encoder = make_column_transformer(
    ( fill_missing_then_one_hot_encoder , cat_columns),
    ( fill_missing_then_StandardScaler, num_columns)
)
data_preprocessoneOnehotPolynomialFeatures = make_column_transformer(
    ( fill_missing_then_one_hot_encoder , cat_columns),
    ( fill_missing_then_StandardScalerPolynomialFeatures, num_columns)
)

# Check


In [ ]:
data_preprocessone_hot_encoder.fit(X_train)
data_preprocessone_hot_encoder.transform(X_train)
data_preprocessone_hot_encoder.transform(X_test)
print("Ok , Every thing is well ")

In [5]:
data_preprocessone_hot_encoder.fit(X_train_condense)
data_preprocessone_hot_encoder.transform(X_train_condense)
data_preprocessone_hot_encoder.transform(X_test_condense)
print("Ok , Every thing is well ")

NameError: name 'data_preprocessone_hot_encoder' is not defined

In [ ]:
data_preprocessone_hot_encoder.fit(X_train_condense2)
data_preprocessone_hot_encoder.transform(X_train_condense2)
data_preprocessone_hot_encoder.transform(X_test_condense2)
print("Ok , Every thing is well ")

# Bin Target :

In [ ]:
train.target.describe()

In [ ]:
bins = [0.140329,7.742071,  8.728634, 10.411992]
# Bin labels
labels1 = [ 'Low', 'Medium', 'High']
trainessai=train.copy()
# Bin the continuous variable ConvertedSalary using these boundaries
trainessai['target_binned'] = pd.cut(trainessai['target'], 
                                         bins=bins,labels=labels1 )

In [ ]:
labels = trainessai['target_binned'].astype('category').cat.categories.tolist()
counts = trainessai['target_binned'].value_counts()
sizes = [counts[var_cat] for var_cat in labels]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=True) #autopct is show the % on plot
ax1.axis('equal')
plt.show()


# t-SNE visualization of high-dimensional data¶

t-SNE intuition t-SNE is super powerful, but do you know exactly when to use it? When you want to visually explore the patterns in a high dimensional dataset. press

In [ ]:
m = TSNE(learning_rate=50)
df_numeric =trainessai.drop(['id','target'], axis=1).iloc[:5000,:]._get_numeric_data()
df_numeric=df_numeric.dropna()
# Fit and transform the t-SNE model on the numeric dataset
tsne_features = m.fit_transform(df_numeric)
%timeit 
print(tsne_features.shape)

In [ ]:
dataa=trainessai.drop(['id','target'], axis=1).iloc[:5000,:]
dataa['x']=tsne_features[:, 0]
dataa['y']=tsne_features[:, 1]
# Color the points according to Army Component
sns.scatterplot(x='x', y='y', hue='target_binned', data=dataa)
# Show the plot
plt.show()

# The curse of dimensionality

In fact, to avoid overfitting the number of observations should increase exponentially with the number of features. Since this becomes really problematic for high dimensional datasets this phenomenon is known as the curse of dimensionality. The solution to this is of course to apply dimensionality reduction.

# Features with missing values or little variance

In [ ]:
# Create the boxplot
trainessai.boxplot()
plt.show()

In [ ]:
# Normalize the data
df_numeric =trainessai.drop(['id','target'], axis=1)._get_numeric_data()
normalized_df = df_numeric/df_numeric.mean()

normalized_df.boxplot()
plt.show()

In [ ]:
trainessai.var()

In [ ]:
# Normalize the data
normalized_df = df_numeric / df_numeric.mean()

# Print the variances of the normalized data
print(normalized_df.var())


## Finding a good variance threshold

Question Inspect the printed variances. If you want to remove the 2 very low variance features. What would be a good variance threshold? Possible Answers
 ==> A threshold of  (0.19) will remove the two low variance features.
## Features with low variance

In the previous exercise you established that 0.19 is a good threshold to filter out low variance features in head_df after normalization. Now use the VarianceThreshold feature selector to remove these features.

In [ ]:
from sklearn.feature_selection import VarianceThreshold

# Create a VarianceThreshold feature selector
sel = VarianceThreshold(threshold=0.19)

# Fit the selector to normalized head_df
sel.fit(df_numeric / df_numeric.mean())

# Create a boolean mask
mask = sel.get_support()

# Apply the mask to create a reduced dataframe
reduced_df = df_numeric.loc[:, mask]

print("Dimensionality reduced from {} to {}.".format(df_numeric.shape[1], reduced_df.shape[1]))

cont7 can be  removed 

# Removing redundant features
## Selecting relevant features
Visually detecting redundant features

### Pairwise correlation

In [ ]:
trainessai.corr()

In [ ]:
import seaborn as sns 
# Create the correlation matrix
corr = trainessai.drop(['id'], axis=1).corr()
cmap = sns.diverging_palette(h_neg=10,  h_pos=240, as_cmap=True)
# Draw the heatmap
sns.heatmap(corr,  cmap=cmap, center=0, linewidths=1, annot=True, fmt=".2f")
plt.show()

In [ ]:
# Generate a mask for the upper triangle 
mask = np.triu(np.ones_like(corr, dtype=bool))
# Add the mask to the heatmap
sns.heatmap(corr, mask=mask, cmap=cmap, center=0, linewidths=1, annot=True, fmt=".2f")
plt.show()

# Removing highly correlated features

In [ ]:
# Calculate the correlation matrix and take the absolute value
corr_matrix = trainessai.drop(['id'], axis=1).corr().abs()

# Create a True/False mask and apply it
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
tri_df = corr_matrix.mask(mask)

# List column names of highly correlated features (r > 0.95)
to_drop = [c for c in tri_df.columns if any(tri_df[c] > 0.4)]

# Drop the features in the to_drop list
reduced_df = trainessai.drop(['id'], axis=1).drop(to_drop, axis=1)
print(to_drop)
print("The reduced dataframe has {} columns.".format(reduced_df.shape[1]))

['cont0', 'cont5', 'cont8', 'cont9', 'cont10']
The reduced dataframe has 21 columns.

#  Automatic Recursive Feature Elimination


In [ ]:
X_train.columns

# Mask1 :

In [ ]:
from sklearn.feature_selection import RFE
# Create the RFE with a LinearRegression estimator and 20 features to select
rfe_LR = RFE(estimator=LinearRegression(), n_features_to_select = 20, verbose=1)

# Fits the eliminator to the data
rfe_LR.fit(data_preprocessOrdinalEncoder.fit_transform(X_train), y_train)

# Print the features and their ranking (high = dropped early on)
print(dict(zip(X.columns, rfe_LR.ranking_)))

# Print the features that are not eliminated
print(X.columns[rfe_LR.support_])

# Calculates the test set accuracy
rmse = mean_squared_error(y_test, rfe_LR.predict(data_preprocessOrdinalEncoder.transform(X_test)), squared=False)
print("{} rmse on test set.".format(rmse))

In [ ]:
mask_LR=rfe_LR.support_
mask_LR

    Index(['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cont0',
       'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9',
       'cont10', 'cont11', 'cont12', 'cont13'],
      dtype='object')
      
    0.7418047219576385 rmse on test set.
    array([ True,  True,  True,  True,  True,  True,  True, False, False,
       False,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

## Mask 2

In [ ]:
from sklearn.feature_selection import RFE
# Create the RFE with a LinearRegression estimator and 20 features to select
rfe_RFR  = RFE(estimator=RandomForestRegressor(n_jobs=-1), n_features_to_select = 20, verbose=1)
# Fits the eliminator to the data
rfe_RFR.fit(data_preprocessOrdinalEncoder.fit_transform(X_train), y_train)
# Print the features and their ranking (high = dropped early on)
print(dict(zip(X.columns, rfe_RFR.ranking_)))
# Print the features that are not eliminated
print(X.columns[rfe_RFR.support_])

In [ ]:
# Calculates the test set accuracy
rmse = mean_squared_error(y_test, rfe_RFR.predict(data_preprocessOrdinalEncoder.transform(X_test)), squared=False)
print("{} rmse on test set.".format(rmse))

In [ ]:
mask_RFR=rfe_RFR.support_
mask_RFR

    0.73964538954687 rmse on test set.
    array([ True, False, False,  True, False,  True, False,  True,  True,
            True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True])

## Mask3

In [ ]:
from sklearn.feature_selection import RFE
# Create the RFE with a LinearRegression estimator and 20 features to select
rfe_GBR = RFE(estimator=GradientBoostingRegressor(), n_features_to_select = 20, verbose=1)

# Fits the eliminator to the data
rfe_GBR.fit(data_preprocessOrdinalEncoder.fit_transform(X_train), y_train)

# Print the features and their ranking (high = dropped early on)
print(dict(zip(X.columns, rfe_GBR.ranking_)))

# Print the features that are not eliminated
print(X.columns[rfe_GBR.support_])

# Calculates the test set accuracy
rmse = mean_squared_error(y_test, rfe_GBR.predict(data_preprocessOrdinalEncoder.transform(X_test)), squared=False)
print("{} rmse on test set.".format(rmse))

In [ ]:
mask_GBR=rfe_GBR.support_
mask_GBR

    Index(['cat1', 'cat3', 'cat6', 'cat7', 'cat8', 'cat9', 'cont0', 'cont1',
           'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9',
           'cont10', 'cont11', 'cont12', 'cont13'],
          dtype='object')
    0.7337589093625022 rmse on test set.
    array([False,  True, False,  True, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

## Mask 4

In [ ]:
from sklearn.feature_selection import RFE
# Create the RFE with a LinearRegression estimator and 20 features to select
rfe_Ridge = RFE(estimator= Ridge(alpha=0.01), n_features_to_select = 20, verbose=1)
# Fits the eliminator to the data
rfe_Ridge.fit(data_preprocessOrdinalEncoder.fit_transform(X_train), y_train)
# Print the features and their ranking (high = dropped early on)
print(dict(zip(X_train.columns, rfe_Ridge.ranking_)))
# Print the features that are not eliminated
print(X.columns[rfe_Ridge.support_])
# Calculates the test set accuracy
rmse = mean_squared_error(y_test, rfe_Ridge.predict(data_preprocessOrdinalEncoder.transform(X_test)), squared=False)
print("{} rmse on test set.".format(rmse))

In [ ]:
mask_Ridge=rfe_Ridge.support_
mask_Ridge

    0.7418047219647409 rmse on test set.
    array([ True,  True,  True,  True,  True,  True,  True, False, False,
           False,  True, False,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True])

## Mask5 

In [ ]:
from sklearn.feature_selection import RFE
# Create the RFE with a LinearRegression estimator and 20 features to select
rfe_Lasso = RFE(estimator= Lasso(), n_features_to_select = 20, verbose=1)
# Fits the eliminator to the data
rfe_Lasso.fit(data_preprocessOrdinalEncoder.fit_transform(X_train), y_train)
# Print the features and their ranking (high = dropped early on)
print(dict(zip(X_train.columns, rfe_Lasso.ranking_)))
# Print the features that are not eliminated
print(X.columns[rfe_Lasso.support_])
# Calculates the test set accuracy
rmse = mean_squared_error(y_test, rfe_Lasso.predict(data_preprocessOrdinalEncoder.transform(X_test)), squared=False)
print("{} rmse on test set.".format(rmse))

In [ ]:
mask_Lasso=rfe_Lasso.support_
mask_Lasso

## Mask6

In [ ]:
from sklearn.feature_selection import RFE
# Create the RFE with a LinearRegression estimator and 20 features to select
rfe_ElasticNet= RFE(estimator= ElasticNet(alpha = 0.01), n_features_to_select = 20, verbose=1)
# Fits the eliminator to the data
rfe_ElasticNet.fit(data_preprocessOrdinalEncoder.fit_transform(X_train), y_train)
# Print the features and their ranking (high = dropped early on)
print(dict(zip(X_train.columns, rfe_ElasticNet.ranking_)))
# Print the features that are not eliminated
print(X.columns[rfe_ElasticNet.support_])
# Calculates the test set accuracy
rmse = mean_squared_error(y_test, rfe_ElasticNet.predict(data_preprocessOrdinalEncoder.transform(X_test)), squared=False)
print("{} rmse on test set.".format(rmse))

In [ ]:
mask_ElasticNet=rfe_ElasticNet.support_
mask_ElasticNet

    0.7424049471095415 rmse on test set.
    array([False,  True, False,  True, False,  True,  True,  True,  True,
            True,  True, False,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True])

# Combining features selector

In [ ]:
# Sum the votes of the three models
votes = np.sum([mask_LR, mask_RFR, mask_GBR,mask_SVR,
                mask_Ridge,mask_Lasso,mask_ElasticNet], axis=0)

# Create a mask for features selected by all 5 models
meta_mask = votes >= 4

# Apply the dimensionality reduction on X
X_reduced = X.loc[:, meta_mask]
print(X.columns)
print(X_reduced.columns)

In [ ]:
temp1=set(X.columns)
s=set(X_reduced.columns)
temp3 = [x for x in temp1 if x not in s]
print("list to drop {}".format(temp3))

list to drop ['cat0', 'cat2', 'cont1', 'cat4'] +

# Features Engineer : 

# Pipe with PolynomialFeatures

In [ ]:
!nvidia-smi

In [ ]:
# importing libraries for polynomial transform
from xgboost import XGBRegressor
from sklearn.decomposition import PCA
XGBR = XGBRegressor(n_estimators = 2000,learning_rate = 0.03,
                    random_state = 22, tree_method = 'gpu_hist', reg_alpha = 10)
from sklearn.preprocessing import PolynomialFeatures
pipepoly = Pipeline([
        ('scaler+ONE+poly', data_preprocessoneOnehotPolynomialFeatures),
        ('classifier', XGBR)])

# Fit the pipeline to the training data
pipepoly.fit(X_train, y_train)
preds_valid = pipepoly.predict(X_test)
print(mean_squared_error(y_test, preds_valid, squared=False))

In [ ]:
#0.725598888349985

## Sparse interation 

In [ ]:
class SparseInteractions(BaseEstimator, TransformerMixin):
    def __init__(self, degree=2, feature_name_separator="_"):
        self.degree = degree
        self.feature_name_separator = feature_name_separator
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        if not sparse.isspmatrix_csc(X):
            X = sparse.csc_matrix(X)
            
        if hasattr(X, "columns"):
            self.orig_col_names = X.columns
        else:
            self.orig_col_names = np.array([str(i) for i in range(X.shape[1])])
            
        spi = self._create_sparse_interactions(X)
        return spi
    
    
    def get_feature_names(self):
        return self.feature_names
    
    def _create_sparse_interactions(self, X):
        out_mat = []
        self.feature_names = self.orig_col_names.tolist()
        
        for sub_degree in range(2, self.degree + 1):
            for col_ixs in combinations(range(X.shape[1]), sub_degree):
                # add name for new column
                name = self.feature_name_separator.join(self.orig_col_names[list(col_ixs)])
                self.feature_names.append(name)
                
                # get column multiplications value
                out = X[:, col_ixs[0]]    
                for j in col_ixs[1:]:
                    out = out.multiply(X[:, j])

                out_mat.append(out)

        return sparse.hstack([X] + out_mat)
# Outlier Handle 
class OutlierReplace(BaseEstimator,TransformerMixin):
    def __init__(self,factor=1.5):
        self.factor = factor

    def outlier_removal(self,X,y=None):
        X = pd.Series(X).copy()
        qmin=X.quantile(0.05)
        qmax=X.quantile(0.95)
        q1 = X.quantile(0.25)
        q3 = X.quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - (self.factor * iqr)
        upper_bound = q3 + (self.factor * iqr)
        #X.loc[((X < lower_bound) | (X > upper_bound))] = np.nan 
        X.loc[X < lower_bound] = qmin
        X.loc[X > upper_bound] = qmax
        return pd.Series(X)

    def fit(self,X,y=None):
        return self

    def transform(self,X,y=None):
        return X.apply(self.outlier_removal) 

In [ ]:
# importing libraries for polynomial transform
from xgboost import XGBRegressor
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
XGBR = XGBRegressor(n_estimators = 2000,learning_rate = 0.03,
                    random_state = 22, tree_method = 'gpu_hist', reg_alpha = 10)
from sklearn.preprocessing import PolynomialFeatures
pipepoly = Pipeline([
        ('scaler+ONE+poly', data_preprocessoneOnehotPolynomialFeatures),
         ('dim_red', SelectKBest(f_regression, k=10)),
         ('interactions', SparseInteractions(degree=2)),
        ('classifier', XGBR)])

# Fit the pipeline to the training data
pipepoly.fit(X_train, y_train)
preds_valid = pipepoly.predict(X_test)
print(mean_squared_error(y_test, preds_valid, squared=False))

In [ ]:
data_preprocessOrdinalEncoderPolynomialFeatures = make_column_transformer(
    ( fill_missing_then_OrdinalEncoder , cat_columns),
    ( fill_missing_then_StandardScalerPolynomialFeatures, num_columns)
)
data_preprocessoneOnehotPolynomialFeatures = make_column_transformer(
    ( fill_missing_then_one_hot_encoder , cat_columns),
    ( fill_missing_then_StandardScalerPolynomialFeatures, num_columns)
)
# importing libraries for polynomial transform
from xgboost import XGBRegressor
from sklearn.decomposition import PCA
XGBR = XGBRegressor(n_estimators = 2000,learning_rate = 0.03,
                    random_state = 22, tree_method = 'gpu_hist', reg_alpha = 10)
from sklearn.preprocessing import PolynomialFeatures
pipepolyordi = Pipeline([
        ('scaler+ONE+poly', data_preprocessOrdinalEncoderPolynomialFeatures),
        ('classifier', XGBR)])

# Fit the pipeline to the training data
pipepolyordi.fit(X_train, y_train)
preds_valid = pipepolyordi.predict(X_test)
print(mean_squared_error(y_test, preds_valid, squared=False))

0.7258725244969303


# Pipe With PCA 

In [ ]:
XGBR = XGBRegressor(n_estimators = 2000,learning_rate = 0.03,
                    random_state = 22, tree_method = 'gpu_hist', reg_alpha = 10)
from sklearn.preprocessing import PolynomialFeatures
pipepolypca = Pipeline([
        ('scaler+ONE+poly', data_preprocessOrdinalEncoderPolynomialFeatures),
        ('interactions', SparseInteractions(degree=2)),
        ('reducer', PCA(n_components=0.9)),
        ('classifier', XGBR)])

# Fit the pipeline to the training data
pipepolypca.fit(X_train, y_train)
preds_valid= pipepolypca.predict(X_test)
print(mean_squared_error(y_test, preds_valid, squared=False))

In [ ]:
# Prints the explained variance ratio and accuracy
print(pipepolypca.steps[1][1].explained_variance_ratio_)

# Chose best Encoder : 
http://www.xavierdupre.fr/app/papierstat/helpsphinx/notebooks/artificiel_category_2.html
https://www.kaggle.com/subinium/11-categorical-encoders-and-benchmark


https://practicaldatascience.co.uk/machine-learning/how-to-use-category-encoders-to-transform-categorical-variables
https://www.kaggle.com/discdiver/category-encoders-examples
https://www.analyticsvidhya.com/blog/2020/08/types-of-categorical-data-encoding/
https://pythonrepo.com/repo/scikit-learn-contrib-category_encoders-python-sklearn-utilities

In [ ]:
encoders = {
    'BackwardDifferenceEncoder': ce.backward_difference.BackwardDifferenceEncoder,
    'BaseNEncoder': ce.basen.BaseNEncoder,
    'BinaryEncoder': ce.binary.BinaryEncoder,
    'CatBoostEncoder': ce.cat_boost.CatBoostEncoder,
    'HashingEncoder': ce.hashing.HashingEncoder,
    'HelmertEncoder': ce.helmert.HelmertEncoder,
    'JamesSteinEncoder': ce.james_stein.JamesSteinEncoder,
    'OneHotEncoder': ce.one_hot.OneHotEncoder,
    'LeaveOneOutEncoder': ce.leave_one_out.LeaveOneOutEncoder,
    'MEstimateEncoder': ce.m_estimate.MEstimateEncoder,
    'OrdinalEncoder': ce.ordinal.OrdinalEncoder,
    'PolynomialEncoder': ce.polynomial.PolynomialEncoder,
    'SumEncoder': ce.sum_coding.SumEncoder,
    'TargetEncoder': ce.target_encoder.TargetEncoder,
    'WOEEncoder': ce.woe.WOEEncoder
}
df_resultsXGBR = pd.DataFrame(columns=['encoder', 'r2', 'mse', 'rmse'])

for key in encoders:
    try :

        categorical_transformer = Pipeline(
            steps=[
                ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('encoder', encoders[key]())
            ]
        )    

        numeric_transformer = Pipeline(
            steps=[
                ('imputer', SimpleImputer(strategy='mean')),
                ('scaler', StandardScaler())
            ]
        )

        preprocessor = ColumnTransformer(
            transformers=[
                ('numerical', numeric_transformer, num_columns),
                ('categorical', categorical_transformer, cat_columns)
            ]
        )

        pipe_XGBR = Pipeline(
            steps=[
                ('preprocessor', preprocessor),
                ('classifier', Ridge(alpha=0.01))
            ]
        )

        modelpipe_XGBR = pipe_XGBR.fit(X_train, y_train)
        y_pred = modelpipe_XGBR.predict(X_test)

        row = {
            'encoder': key,
            'r2': modelpipe_XGBR.score(X_test, y_test),
            'mse': mean_squared_error(y_test, y_pred, squared=True),
            'rmse': mean_squared_error(y_test, y_pred,squared=False)
        }
        df_resultsXGBR = df_resultsXGBR.append(row, ignore_index=True)
    except :
        raw={
            'encoder': key,
            'r2': np.nan,
            'mse': np.nan,
            'rmse': np.nan
        }
        df_resultsXGBR = df_resultsXGBR.append(row, ignore_index=True)
    

In [ ]:
df_resultsXGBR.head(20).sort_values(by='rmse')

lR :

        encoder 	r2 	mse 	rmse
        1 	BaseNEncoder 	0.023040 	0.548627 	0.740694
        2 	BinaryEncoder 	0.023040 	0.548627 	0.740694
        0 	BackwardDifferenceEncoder 	0.022985 	0.548658 	0.740714
        7 	OneHotEncoder 	0.022985 	0.548658 	0.740714
        11 	PolynomialEncoder 	0.022985 	0.548658 	0.740714
        12 	SumEncoder 	0.022985 	0.548658 	0.740714
        5 	HelmertEncoder 	0.022985 	0.548658 	0.740714
        9 	MEstimateEncoder 	0.022730 	0.548801 	0.740811
        13 	TargetEncoder 	0.022728 	0.548802 	0.740812
        14 	TargetEncoder 	0.022728 	0.548802 	0.740812
        6 	JamesSteinEncoder 	0.022727 	0.548803 	0.740812
        8 	LeaveOneOutEncoder 	0.022723 	0.548805 	0.740814
        3 	CatBoostEncoder 	0.022414 	0.548978 	0.740931
        10 	OrdinalEncoder 	0.020120 	0.550266 	0.741799
        4 	HashingEncoder 	0.018452 	0.551203 	0.742431

In [ ]:
XGBR = XGBRegressor(n_estimators = 2000,learning_rate = 0.03,
                    random_state = 22, tree_method = 'gpu_hist', reg_alpha = 10)
encoders = {
    'BackwardDifferenceEncoder': ce.backward_difference.BackwardDifferenceEncoder,
    'BaseNEncoder': ce.basen.BaseNEncoder,
    'BinaryEncoder': ce.binary.BinaryEncoder,
    'CatBoostEncoder': ce.cat_boost.CatBoostEncoder,
    'HashingEncoder': ce.hashing.HashingEncoder,
    'HelmertEncoder': ce.helmert.HelmertEncoder,
    'JamesSteinEncoder': ce.james_stein.JamesSteinEncoder,
    'OneHotEncoder': ce.one_hot.OneHotEncoder,
    'LeaveOneOutEncoder': ce.leave_one_out.LeaveOneOutEncoder,
    'MEstimateEncoder': ce.m_estimate.MEstimateEncoder,
    'OrdinalEncoder': ce.ordinal.OrdinalEncoder,
    'PolynomialEncoder': ce.polynomial.PolynomialEncoder,
    'SumEncoder': ce.sum_coding.SumEncoder,
    'TargetEncoder': ce.target_encoder.TargetEncoder,
    'WOEEncoder': ce.woe.WOEEncoder
}
df_resultsXGBR = pd.DataFrame(columns=['encoder', 'r2', 'mse', 'rmse'])

for key in encoders:
    try :

        categorical_transformer = Pipeline(
            steps=[
                ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('encoder', encoders[key]())
            ]
        )    

        numeric_transformer = Pipeline(
            steps=[
                ('imputer', SimpleImputer(strategy='mean')),
                ('scaler', StandardScaler())
            ]
        )

        preprocessor = ColumnTransformer(
            transformers=[
                ('numerical', numeric_transformer, num_columns),
                ('categorical', categorical_transformer, cat_columns)
            ]
        )

        pipe_XGBR = Pipeline(
            steps=[
                ('preprocessor', preprocessor),
                ('classifier',XGBR)
            ]
        )

        modelpipe_XGBR = pipe_XGBR.fit(X_train, y_train)
        y_pred = modelpipe_XGBR.predict(X_test)

        row = {
            'encoder': key,
            'r2': modelpipe_XGBR.score(X_test, y_test),
            'mse': mean_squared_error(y_test, y_pred, squared=True),
            'rmse': mean_squared_error(y_test, y_pred,squared=False)
        }
        df_resultsXGBR = df_resultsXGBR.append(row, ignore_index=True)
    except :
        raw={
            'encoder': key,
            'r2': np.nan,
            'mse': np.nan,
            'rmse': np.nan
        }
        df_resultsXGBR = df_resultsXGBR.append(row, ignore_index=True)
    

In [ ]:
df_resultsXGBR.head(20).sort_values(by='rmse')

XGBR :

    encoder 	r2 	mse 	rmse
    12 	SumEncoder 	0.067784 	0.523500 	0.723533
    6 	JamesSteinEncoder 	0.067728 	0.523532 	0.723555
    9 	MEstimateEncoder 	0.067617 	0.523594 	0.723598
    13 	TargetEncoder 	0.067617 	0.523594 	0.723598
    14 	TargetEncoder 	0.067617 	0.523594 	0.723598
    7 	OneHotEncoder 	0.067450 	0.523688 	0.723663
    5 	HelmertEncoder 	0.067349 	0.523744 	0.723702
    11 	PolynomialEncoder 	0.067289 	0.523778 	0.723725
    10 	OrdinalEncoder 	0.067080 	0.523895 	0.723806
    0 	BackwardDifferenceEncoder 	0.066829 	0.524036 	0.723904
    3 	CatBoostEncoder 	0.066808 	0.524048 	0.723912
    1 	BaseNEncoder 	0.066565 	0.524185 	0.724006
    2 	BinaryEncoder 	0.066565 	0.524185 	0.724006
    4 	HashingEncoder 	0.063058 	0.526154 	0.725365
    8 	LeaveOneOutEncoder 	0.000761 	0.561138 	0.74909


In [ ]:
Catmodel = CatBoostRegressor(iterations=6800,
                          learning_rate=0.93,
                          loss_function="RMSE",
                          random_state=42,
                          verbose=0,
                          thread_count=4,
                          depth=1,
                          task_type="GPU",
                          l2_leaf_reg=3.28,
                         )
encoders = {
    'BackwardDifferenceEncoder': ce.backward_difference.BackwardDifferenceEncoder,
    'BaseNEncoder': ce.basen.BaseNEncoder,
    'BinaryEncoder': ce.binary.BinaryEncoder,
    'CatBoostEncoder': ce.cat_boost.CatBoostEncoder,
    'HashingEncoder': ce.hashing.HashingEncoder,
    'HelmertEncoder': ce.helmert.HelmertEncoder,
    'JamesSteinEncoder': ce.james_stein.JamesSteinEncoder,
    'OneHotEncoder': ce.one_hot.OneHotEncoder,
    'LeaveOneOutEncoder': ce.leave_one_out.LeaveOneOutEncoder,
    'MEstimateEncoder': ce.m_estimate.MEstimateEncoder,
    'OrdinalEncoder': ce.ordinal.OrdinalEncoder,
    'PolynomialEncoder': ce.polynomial.PolynomialEncoder,
    'SumEncoder': ce.sum_coding.SumEncoder,
    'TargetEncoder': ce.target_encoder.TargetEncoder,
    'WOEEncoder': ce.woe.WOEEncoder
}
df_resultsCat = pd.DataFrame(columns=['encoder', 'r2', 'mse', 'rmse'])

for key in encoders:
    try:
        categorical_transformer = Pipeline(
            steps=[
                ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('encoder', encoders[key]())
            ]
        )    

        numeric_transformer = Pipeline(
            steps=[
                ('imputer', SimpleImputer(strategy='mean')),
                ('scaler', StandardScaler())
            ]
        )

        preprocessor = ColumnTransformer(
            transformers=[
                ('numerical', numeric_transformer, num_columns),
                ('categorical', categorical_transformer, cat_columns)
            ]
        )

        pipe_Cat = Pipeline(
            steps=[
                ('preprocessor', preprocessor),
                ('classifier', Catmodel)
            ]
        )

        modelpipe_Cat = pipe_Cat.fit(X_train, y_train)
        y_pred = modelpipe_Cat.predict(X_test)

        row = {'encoder': key,
            'r2': modelpipe_Cat.score(X_test, y_test),
            'mse': mean_squared_error(y_test, y_pred, squared=True),
            'rmse': mean_squared_error(y_test, y_pred,squared=False)
        }
        df_resultsCat = df_resultsCat.append(row, ignore_index=True)
    except:
        raw={'encoder': key,
        'r2': np.nan,
        'mse': np.nan,
        'rmse': np.nan}
        df_resultsCat = df_resultsCat.append(row, ignore_index=True)

In [ ]:
df_resultsCat.head(20).sort_values(by='rmse')

    OrderedDict([('colsample_bytree', 0.4481260135905676),
                 ('learning_rate', 0.01),
                 ('max_bin', 594),
                 ('max_depth', 11),
                 ('min_child_samples', 83),
                 ('min_child_weight', 8.65733341724971),
                 ('n_estimators', 2693),
                 ('num_leaves', 512),
                 ('objective', 'tweedie'),
                 ('reg_alpha', 0.0049073005447637175),
                 ('reg_lambda', 1e-09),
                 ('reg_sqrt', True),
                 ('subsample', 0.4611042237962284),
                  ('subsample_freq', 2)])
      params = {
     'max_bin': 500,
    'feature_fraction': 0.78,
    'bagging_fraction': 0.78,
    'objective': 'regression',
    'max_depth': -1,
    'learning_rate': 0.01,
    "bagging_seed": 42,
    'random_state': 42,
    "metric": 'mse',
    "verbosity": -1,
}       


        params_lgb = {
            'application': 'regression',
            'metric': 'rmse',
            'learning_rate': 0.01,
            'boosting': 'gbdt',
            'verbose': -1,
            'num_leaves': 293, 
            'feature_fraction': 0.7649753076229889, 
            'bagging_fraction': 0.7888028266800043, 
            'lambda_l1': 7.882036517237124e-05, 
            'lambda_l2': 1.2301397753601606e-08, 
            'min_split_gain': 0.3615899594462653
        }


             
                params = {
    "objective": "rmse",
    "metric": "rmse",
    "boosting_type": 'gbdt', #"gbdt", "dart"
    #"xgboost_dart_mode": True,
    'device_type': 'cpu', #'gpu', 'cuda'
    #'drop_rate': 0.5, #For dart type only
    'seed': seed,
    #'deterministic': True,
    'tree_learner': 'data',
    'num_threads': 4,
    'max_bin': 512,
    'max_depth': 20,
    'num_leaves': 250,
    'early_stopping_rounds': 500,
    'lambda_l1': 1.75,
    'lambda_l2': 3.25,
    'cat_l2': 1.25,
    'learning_rate': 0.005,
    'feature_fraction': 0.25,
    "feature_fraction_seed": seed,
    'bagging_fraction': 0.75,
    'bagging_freq': 50,
    'bagging_seed': seed,
    'force_col_wise': True,
    #'min_data_in_leaf': 27,
    'path_smooth': 2.5,
    'cat_smooth': 2.5,
  }
                
       lgbm_params = {
        "random_state": 2021,
        "metric": "rmse",
        "n_jobs": -1,
        "early_stopping_round": 350,
        "reg_lambda": 0.01,
        "colsample_bytree": 0.33,
        "learning_rate": 0.005,
        "max_depth": 65,
        "num_leaves": 250,
        "min_child_samples": 100,
        "n_estimators": 2000,
        "cat_smooth": 50,
        "max_bin": 537,
        "min_data_per_group": 117,
        "bagging_freq": 25,
        "bagging_fraction": 0.6709049555262285,
        "cat_l2": 7.5586732660804445,
    }
              
# Transferring the best parameters to our basic regressor
reg = lgb.LGBMRegressor(boosting_type='gbdt',
                        n_jobs=-1, 
                        verbose=-1,
                        random_state=0,
                         **best_params)

In [ ]:
import lightgbm as lgbm
seed = np.random.randint(1, 1e5)
lgbm_params = {
        "random_state": 2021,
        "metric": "rmse",
        "reg_lambda": 0.01,
        "colsample_bytree": 0.33,
        "learning_rate": 0.005,
        "max_depth": 65,
        "num_leaves": 250,
        "min_child_samples": 100,
        "n_estimators": 2000,
        "cat_smooth": 50,
        "min_data_per_group": 117,
        "cat_l2": 7.5586732660804445,
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0}
              
# Transferring the best parameters to our basic regressor
lgbmreg = lgbm.LGBMRegressor(boosting_type='gbdt',
                        n_jobs=-1, 
                        verbose=-1,
                         **lgbm_params)
encoders = {
    'BackwardDifferenceEncoder': ce.backward_difference.BackwardDifferenceEncoder,
    'BaseNEncoder': ce.basen.BaseNEncoder,
    'BinaryEncoder': ce.binary.BinaryEncoder,
    'CatBoostEncoder': ce.cat_boost.CatBoostEncoder,
    'HashingEncoder': ce.hashing.HashingEncoder,
    'HelmertEncoder': ce.helmert.HelmertEncoder,
    'JamesSteinEncoder': ce.james_stein.JamesSteinEncoder,
    'OneHotEncoder': ce.one_hot.OneHotEncoder,
    'LeaveOneOutEncoder': ce.leave_one_out.LeaveOneOutEncoder,
    'MEstimateEncoder': ce.m_estimate.MEstimateEncoder,
    'OrdinalEncoder': ce.ordinal.OrdinalEncoder,
    'PolynomialEncoder': ce.polynomial.PolynomialEncoder,
    'SumEncoder': ce.sum_coding.SumEncoder,
    'TargetEncoder': ce.target_encoder.TargetEncoder,
    'WOEEncoder': ce.woe.WOEEncoder
}
df_resultslgbmreg = pd.DataFrame(columns=['encoder', 'r2', 'mse', 'rmse'])

for key in encoders:
    try:
        categorical_transformer = Pipeline(
        steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
            ('encoder', encoders[key]())
        ]
        )    

        numeric_transformer = Pipeline(
        steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]
        )

        preprocessor = ColumnTransformer(
        transformers=[
            ('numerical', numeric_transformer, num_columns),
            ('categorical', categorical_transformer, cat_columns)
        ]
        )

        pipe_lgbmreg = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('classifier', lgbmreg)
        ]
        )

        modelpipe_lgbmreg = pipe_lgbmreg.fit(X_train, y_train)
        y_pred = modelpipe_lgbmreg.predict(X_test)

        row = {'encoder': key,
        'r2': modelpipe_lgbmreg.score(X_test, y_test),
        'mse': mean_squared_error(y_test, y_pred, squared=True),
        'rmse': mean_squared_error(y_test, y_pred,squared=False)
        }
        df_resultslgbmreg = df_resultslgbmreg.append(row, ignore_index=True)
    except:
        raw={'encoder': key,
        'r2': np.nan,
        'mse': np.nan,
        'rmse': np.nan}
        df_resultslgbmreg = df_resultslgbmreg.append(row, ignore_index=True)   

In [ ]:

df_resultslgbmreg.head(20).sort_values(by='rmse')

        LGBM:
        
       encoder 	r2 	mse 	rmse
        12 	SumEncoder 	0.068020 	0.523367 	0.723441
        2 	BinaryEncoder 	0.067894 	0.523438 	0.723490
        1 	BaseNEncoder 	0.067690 	0.523553 	0.723570
        7 	OneHotEncoder 	0.067670 	0.523564 	0.723577
        5 	HelmertEncoder 	0.067628 	0.523588 	0.723594
        0 	BackwardDifferenceEncoder 	0.067431 	0.523698 	0.723670
        11 	PolynomialEncoder 	0.066782 	0.524063 	0.723922
        13 	TargetEncoder 	0.065995 	0.524505 	0.724227
        14 	TargetEncoder 	0.065995 	0.524505 	0.724227
        9 	MEstimateEncoder 	0.065979 	0.524514 	0.724233
        10 	OrdinalEncoder 	0.065936 	0.524538 	0.724250
        6 	JamesSteinEncoder 	0.065931 	0.524541 	0.724252
        3 	CatBoostEncoder 	0.064839 	0.525154 	0.724675
        4 	HashingEncoder 	0.063647 	0.525823 	0.725137
        8 	LeaveOneOutEncoder 	0.000478 	0.561297 	0.749197

# COnclusion : 
features to reduce : 
cont7 can be removed  : low variance 
list to drop ['cat0', 'cat2', 'cont1', 'cat4'] less voted by algho .
i shoyld take a view in this notebook :

https://www.kaggle.com/dwin183287/30-days-of-ml-eda
https://www.kaggle.com/vipin20/getting-start-30-days-ml
